In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

from transformers import AutoTokenizer, AutoModel

C:\Users\Parth\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Dataset

In [2]:
import pandas as pd

# Define correct column names
column_names = ["sentence", "label"]

def load_dataset1(train_url, test_url):
    train_df = pd.read_csv(train_url, sep='\t', names=column_names, header=None)
    test_df = pd.read_csv(test_url, sep='\t', names=column_names, header=None)
    return train_df, test_df

# URLs for SST2 dataset
train_url = "https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv"
test_url = "https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/test.tsv"

# Load datasets
train_df, test_df = load_dataset1(train_url, test_url)

# Display first few rows to confirm correct loading
print(train_df.head())


# Use the 'sentence' column since SST2 uses it instead of 'text'
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['sentence'], train_df['label'], test_size=0.2, random_state=42
)

print(f"Number of training samples: {len(train_texts)}")
print(f"Number of validation samples: {len(val_texts)}")
print(f"Number of testing samples: {len(test_df)}")

                                            sentence  label
0  a stirring , funny and finally transporting re...      1
1  apparently reassembled from the cutting room f...      0
2  they presume their audience wo n't sit still f...      0
3  this is a visually stunning rumination on love...      1
4  jonathan parker 's bartleby should have been t...      1
Number of training samples: 5536
Number of validation samples: 1384
Number of testing samples: 1821


# MLP Model

In [3]:
import torch
import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_size):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)  # Output layer for binary classification
        )

    def forward(self, x):
        return self.model(x)

# Example input size (to be updated based on feature extraction method)
input_size = 10000  # Adjusted for Bag-of-Words

# Initialize model
model = MLPClassifier(input_size)
print(model)

# Count trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Trainable Parameters: {total_params}")


MLPClassifier(
  (model): Sequential(
    (0): Linear(in_features=10000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=2, bias=True)
  )
)
Total Trainable Parameters: 5293122


# Case 1: Implement Bag-of-Words (BoW)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# Create Bag-of-Words vectorizer
vectorizer = CountVectorizer(max_features=10000)

# Fit and transform the text data
X_train_bow = vectorizer.fit_transform(train_texts).toarray()
X_val_bow = vectorizer.transform(val_texts).toarray()

# Convert to PyTorch tensors
X_train_bow = torch.tensor(X_train_bow, dtype=torch.float32)
X_val_bow = torch.tensor(X_val_bow, dtype=torch.float32)
y_train = torch.tensor(train_labels.values, dtype=torch.long)
y_val = torch.tensor(val_labels.values, dtype=torch.long)

print(f"BoW Feature Shape: {X_train_bow.shape}")


BoW Feature Shape: torch.Size([5536, 10000])


#  Case 2: Implement LLaMA-3.1 Embeddings

In [12]:
!pip install -U "huggingface_hub[cli]"

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/469.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/469.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
      --------------------------------------- 10.2/469.0 kB ? eta -:--:--
     --- ----------------------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\users\\parth\\appdata\\roaming\\python\\python310\\scripts\\huggingface-cli.exe'
Check the permissions.


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:

!huggingface-cli login

^C


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from huggingface_hub import login

# Login to Hugging Face (Only needed once per session)
login(token="YAY TOKEN")

class LLaMaEmbeddings:
    def __init__(self, model_name='meta-llama/Llama-3.1-8B', device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, use_auth_token=True)
        self.model = AutoModel.from_pretrained(self.model_name, use_auth_token=True).to(self.device)
        """THIS WORKS THE PROBLEM IS WITH THE ACESS OF 'meta-llama/Llama-3.1-8B'
        model_name = "bert-base-uncased"  # Publicly available model for testing
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
"""
        self.embedding_size = self.model.config.hidden_size
        self.model_loaded = True

# Test the updated embedding function
llama_embedder = LLaMaEmbeddings()
sample_texts = ["This movie was amazing!", "The plot was boring."]
embeddings = llama_embedder.get_embeddings(sample_texts)
print("Embedding shape:", embeddings.shape)


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like meta-llama/Llama-3.1-8B is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

# Training the Model (BoW and LLaMA in Parallel)

In [ ]:
import torch.optim as optim

# Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

# Select dataset representation (BoW or LLaMA)
X_train = X_train_bow  # Change to X_train_llama for LLaMA embeddings
X_val = X_val_bow  # Change to X_val_llama for LLaMA embeddings

# Training Loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)
    
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")




# Save and Load Checkpoints

In [ ]:
# Save final model checkpoint
final_checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'hyperparameters': {
        'input_size': input_size,
        'hidden_sizes': [512, 256, 128, 64],  # MLP architecture
        'output_size': 2,  # Binary classification
        'learning_rate': 0.0001,
        'epochs': epochs
    },
    'training_history': {
        'initial_task': 'SST-2',
        'transfer_task': 'IMDB',
        'sst2_performance': {
            'final_loss': loss.item(),
            'final_accuracy': accuracy
        },
        'imdb_performance': {
            'final_loss': loss.item(),  # Update with IMDB results after training
            'final_accuracy': accuracy  # Update with IMDB results
        }
    }
}

torch.save(final_checkpoint, 'mlp_text_classification_checkpoint.pth')
print("\nTraining complete! Final model checkpoint saved.")


# Load and Prepare the IMDB Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the IMDB dataset (Note: this dataset contains a "review" column and a "sentiment" column)
imdb_url = "https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv"
imdb_df = pd.read_csv(imdb_url)

# Convert sentiment to numerical labels (assuming 'positive' and 'negative')
imdb_df['label'] = imdb_df['sentiment'].apply(lambda x: 1 if x.lower() == 'positive' else 0)

# Split into training and validation sets (80% training, 20% validation)
imdb_train_df, imdb_val_df = train_test_split(imdb_df, test_size=0.2, random_state=42)

print(f"IMDB training samples: {len(imdb_train_df)}")
print(f"IMDB validation samples: {len(imdb_val_df)}")


#  Feature Extraction and Tensor Conversion

In [ ]:
import torch

# Extract the text and labels from the IMDB training and validation sets
imdb_train_texts = imdb_train_df['review']
imdb_train_labels = imdb_train_df['label']
imdb_val_texts = imdb_val_df['review']
imdb_val_labels = imdb_val_df['label']

# Use the same CountVectorizer (Bag-of-Words) that was fit on the SST-2 dataset
# (Assuming 'vectorizer' has been previously defined and fitted on the training texts of SST-2)
X_train_imdb = vectorizer.transform(imdb_train_texts).toarray()
X_val_imdb = vectorizer.transform(imdb_val_texts).toarray()

# Convert features and labels to PyTorch tensors
X_train_imdb = torch.tensor(X_train_imdb, dtype=torch.float32)
X_val_imdb = torch.tensor(X_val_imdb, dtype=torch.float32)
y_train_imdb = torch.tensor(imdb_train_labels.values, dtype=torch.long)
y_val_imdb = torch.tensor(imdb_val_labels.values, dtype=torch.long)

print(f"IMDB BoW Training Features Shape: {X_train_imdb.shape}")
print(f"IMDB BoW Validation Features Shape: {X_val_imdb.shape}")


# Continual Learning – Fine-tune on IMDB Dataset

In [ ]:
import torch.optim as optim
import torch.nn as nn

# Define or re-use your model, criterion, and optimizer.
# (Assuming 'model' is already defined and was trained on SST-2)
# For continual learning, you might use a smaller learning rate.
transfer_lr = 0.0001
optimizer = optim.Adam(model.parameters(), lr=transfer_lr)
criterion = nn.CrossEntropyLoss()
epochs = 10  # or as required

# Fine-tune the model on the IMDB training set
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_imdb)
    loss = criterion(outputs, y_train_imdb)
    loss.backward()
    optimizer.step()
    
    print(f"IMDB Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate on the IMDB validation set
model.eval()
with torch.no_grad():
    val_outputs = model(X_val_imdb)
    val_loss = criterion(val_outputs, y_val_imdb)
    val_preds = val_outputs.argmax(dim=1)
    accuracy = (val_preds == y_val_imdb).float().mean().item()

print(f"\nIMDB Validation Loss: {val_loss.item():.4f}, Accuracy: {accuracy:.4f}")


# Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions
model.eval()
with torch.no_grad():
    val_preds = model(X_val).argmax(dim=1)

# Compute Accuracy
accuracy = accuracy_score(y_val, val_preds)
print(f"Validation Accuracy: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_val, val_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


# Visualization (TensorBoard Integration)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("runs/text_classification")

for epoch in range(epochs):
    writer.add_scalar("Loss/train", loss.item(), epoch)
    writer.add_scalar("Loss/validation", val_loss.item(), epoch)

writer.close()
print("TensorBoard logs saved.")
